# HyDE - Hypothetical Document Embeddings

- [HyDE: Precise Zero-Shot Dense Retrieval without Relevance Labels](https://github.com/texttron/hyde)
- HyDE creates a "Hypothetical" answer with the LLM and then embeds that for search

![](https://github.com/texttron/hyde/raw/main/approach.png)

# 0. Setup

In [54]:
!pip -q install -U boto3 awscli langchain pypdf

In [28]:
import boto3
from langchain.llms.bedrock import Bedrock
from langchain.embeddings import BedrockEmbeddings
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.document_loaders import TextLoader
import langchain

In [3]:
profile_name = None
region = 'us-east-1'

In [4]:
# modelId = 'anthropic.claude-instant-v1'
modelId = 'anthropic.claude-v2'

In [5]:
session = boto3.Session(
    profile_name=profile_name,
    region_name=region,
)
bedrock = session.client(service_name='bedrock-runtime')

## Embeddings

- bedrock embedding model - amazon.titan-embed-text-v1
- 일반적으로는 SOTA 인 baai 의 [bge](https://python.langchain.com/docs/integrations/text_embedding/bge_huggingface) 씀.

In [8]:
llm = Bedrock(
    model_id=modelId,
    client=bedrock,
    model_kwargs={
        'max_tokens_to_sample': 1024
    },
)
bedrock_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1",
    client=bedrock,
)

In [9]:
# Load with `web_search` prompt
embeddings = HypotheticalDocumentEmbedder.from_llm(
    llm,
    bedrock_embeddings,
    prompt_key="web_search",
)

In [10]:
embeddings.llm_chain.prompt

PromptTemplate(input_variables=['QUESTION'], template='Please write a passage to answer the question \nQuestion: {QUESTION}\nPassage:')

In [11]:
langchain.debug = True

# 1. Run query

In [12]:
result = embeddings.embed_query("What items does McDonalds make?")

[llm/start] [1:llm:Bedrock] Entering LLM run with input:
{
  "prompts": [
    "Please write a passage to answer the question \nQuestion: What items does McDonalds make?\nPassage:"
  ]
}
[llm/end] [1:llm:Bedrock] [9.52s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Here is a passage answering the question \"What items does McDonalds make?\":\n\nMcDonald's is known for making a wide variety of fast food items. Some of their most popular menu items include hamburgers, cheeseburgers, Big Macs, Quarter Pounders, Chicken McNuggets, Filet-O-Fish sandwiches, french fries, milkshakes, sodas, coffee, breakfast sandwiches, and desserts like apple pies and McFlurries. McDonald's hamburgers are made with 100% beef patties that are cooked and prepared on a hot grill. Their buns and sandwich ingredients like cheese, lettuce, tomato, onion, pickles, and condiments are assembled on the sandwiches to customer specifications. The french fries are made from potatoes th

# 2. Custom prompt

In [13]:
prompt_template = """
Please answer the user's question as a single food item.
Question: {QUESTION}
Answer: """.strip()
prompt = PromptTemplate(
    input_variables=["QUESTION"],
    template=prompt_template,
)

In [14]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [15]:
embeddings = HypotheticalDocumentEmbedder(
    llm_chain=llm_chain,
    base_embeddings=bedrock_embeddings,
)

In [16]:
query = "What is is McDonalds best selling item?"
result = embeddings.embed_query(query)

[llm/start] [1:llm:Bedrock] Entering LLM run with input:
{
  "prompts": [
    "Please answer the user's question as a single food item.\nQuestion: What is is McDonalds best selling item?\nAnswer:"
  ]
}
[llm/end] [1:llm:Bedrock] [2.56s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Big Mac",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}


# 3. Advanced usage

- AWS IoT Provisioning Whitepaper [다운로드](https://docs.aws.amazon.com/pdfs/whitepapers/latest/device-manufacturing-provisioning/device-manufacturing-provisioning.pdf#device-manufacturing-provisioning)
- 간접적인 질문을 통해 HyDE 사용

In [18]:
loader = PyPDFLoader("iot.pdf")
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = loader.load_and_split(text_splitter=text_splitter)

In [19]:
len(texts)

36

In [20]:
texts[0]

Document(page_content='Device Manufacturing and \nProvisioning with X.509 \nCertiﬁcates in AWS IoT Core\nAWS Whitepaper', metadata={'source': 'iot.pdf', 'page': 0})

In [21]:
prompt_template = """
Please answer the user's question as related to Internet of things provisioning.
Question: {QUESTION}
Answer: """.strip()
prompt = PromptTemplate(
    input_variables=["QUESTION"],
    template=prompt_template,
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [22]:
embeddings = HypotheticalDocumentEmbedder(
    llm_chain=llm_chain,
    base_embeddings=bedrock_embeddings,
)

In [23]:
%%time

docsearch = Chroma.from_documents(texts, embeddings)

CPU times: user 347 ms, sys: 59 ms, total: 406 ms
Wall time: 13.7 s


- nosql 이라는 단어는 문서에 등장하지 않지만, nosql 데이터베이스를 언급하고 mongodb 가 AGPL 사용하고 있다고 알려준다.

In [24]:
query = "What is the best way to provision device when I can not put certificates into my devices?"
docs = docsearch.similarity_search(query)

[llm/start] [1:llm:Bedrock] Entering LLM run with input:
{
  "prompts": [
    "Please answer the user's question as related to Internet of things provisioning.\nQuestion: What is the best way to provision device when I can not put certificates into my devices?\nAnswer:"
  ]
}
[llm/end] [1:llm:Bedrock] [14.48s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Here are a few recommendations for provisioning internet of things (IoT) devices without being able to put certificates on them:\n\n- Use symmetric key encryption - Have a pre-shared key that is used to encrypt communication between the device and IoT platform. The key can be hardcoded into the device firmware and configured on the server side. This avoids the need for certificates.\n\n- Use token-based authentication - The IoT platform can issue a token to an unprovisioned device the first time it connects. This token can then be used for authentication until the device is fully provisioned. The to

In [25]:
print(docs[0].page_content)

Device Manufacturing and Provisioning with X.509 
Certiﬁcates in AWS IoT Core AWS Whitepaper
Just-in-Time Provisioning
Provisioning identity in AWS IoT 
Core for device connections
AWS IoT Core has diﬀerent options to provision and onboard a large number of devices, depending on 
the capabilities of the device, and if the devices have their unique X.509 certiﬁcate and private keys on 
them before being sold to the end customer.
If the manufacturing chain allows the device maker to provision unique credentials into the device at 
manufacturing time or in distribution, device makers can use Just in Time Provisioning (JITP), Just in 
Time Registration (JITR), or Multi-Account Registration (MAR).
If it is not possible to establish unique credentials on the device before it is sold to the end customers, 
device makers may use Fleet Provisioning to onboard their devices.
Just-in-Time Provisioning
Devices that use Just-in-Time Provisioning (JITP) have certiﬁcates and private keys present on t

In [26]:
retriever = docsearch.as_retriever(search_kwargs={"k": 5})
retriever.search_type

'similarity'

In [29]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True,
)

In [30]:
query = "What is the best way to provision device when I can not put certificates into my devices?"
llm_response = qa_chain(query)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is the best way to provision device when I can not put certificates into my devices?"
}
[llm/start] [1:llm:Bedrock] Entering LLM run with input:
{
  "prompts": [
    "Please answer the user's question as related to Internet of things provisioning.\nQuestion: What is the best way to provision device when I can not put certificates into my devices?\nAnswer:"
  ]
}
[llm/end] [1:llm:Bedrock] [17.33s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Here are a few potential ways to provision devices that cannot store certificates for Internet of Things (IoT):\n\n- Use symmetric key encryption instead of certificates for authentication. Generate a unique symmetric key for each device and store it securely on the server side. The device can then use this key to authenticate to the server.\n\n- Use a Trust On First Use (TOFU) model. The first time a device connects, its creden

In [31]:
print(llm_response['result'])

 Based on the context provided, if you are unable to put certificates into your devices before they are sold to customers, the best option for provisioning devices in AWS IoT Core is likely fleet provisioning. 

The whitepaper mentions that fleet provisioning provides two ways to provision devices with unique credentials after they are delivered to end customers:

1. Fleet provisioning by trusted user: An installer uses a mobile app to generate temporary credentials that are delivered to the unprovisioned device. The device uses these to connect to AWS IoT and exchange them for a unique certificate and key.

2. Fleet provisioning by claim: Devices can claim a bulk allotment of unique credentials that were generated in advance by the device maker. The credentials are not bound to a specific device until claimed.

So in summary, fleet provisioning allows provisioning devices that do not have unique credentials pre-loaded, by generating and delivering credentials after the device is alrea